In [27]:
import time
import numpy as np
import sqlite3
import pandas as pd

# Import queries, schema info & clean entries

queryDocPath = "/Users/wayne/Desktop/SQL/queryDoc.xlsx"
def schemaInfoImport(filepath):
    importDataframe = pd.read_excel(filepath, sheet_name="query-jupyter", header=0)
    global sql_list
    global table_list
    global group_by_list
    sql_list        = importDataframe['QUERY'].tolist()
    table_list      = importDataframe['Table Name'].tolist()
    group_by_list   = importDataframe['Group By'].tolist()

def attributeCleaner(groupbyList):
    for m in range(len(groupbyList)):
        group_by_list[m] = group_by_list[m].split(',')
        for n in range(len(groupbyList[m])):
            groupbyList[m][n] = groupbyList[m][n].lstrip(' ')
            groupbyList[m][n] = groupbyList[m][n].replace(u'\xa0', u'')
            
schemaInfoImport(queryDocPath)            
attributeCleaner(group_by_list)

print(sql_list[0])

SELECT entity_identifier, group_by1, count(*) as COUNT, sum(balance_in_ccy1) as balance_in_ccy1, sum(balance_in_ccy2) as balance_in_ccy2 FROM FINOTC_SRC_DATA1 GROUP BY entity_identifier, group_by1


In [35]:
# Import both environment tables (ENV1 from db, ENV2 from csv)

con = sqlite3.connect("/Users/wayne/DataGripProjects/FIN_DATA/identifier.sqlite")

tables_checked = []
def ENV1_import(query, table, groupby):
    print(table)
    tables_checked.append(table) 
    global ENV1_raw
    ENV1_raw     = pd.read_sql_query(query, con)
    column_list  = (ENV1_raw.columns).tolist() 
    
    ENV1_pivot         = pd.pivot_table(ENV1_raw,index=groupby,  values = column_list, aggfunc='sum', fill_value=0)
    ENV1_pivot_columns = ENV1_pivot.columns
#     ENV1_pivot         = ENV1_pivot.append(ENV1_raw[ENV1_pivot_columns].sum().rename('Grand Total')).fillna('')
    return(ENV1_pivot)

def ENV2_import(csv_filepath, table, groupby):
    ENV2_filepath      = ("/Users/wayne/Desktop/SQL/csv/"+ table + ".csv")
    global ENV2_raw
    ENV2_raw           = pd.read_csv(ENV2_filepath, header=0)
    column_list        = (ENV2_raw.columns).tolist()
    
    ENV2_pivot         = pd.pivot_table(ENV2_raw,index=groupby, values = column_list, aggfunc='sum')
    ENV2_pivot_columns = ENV2_pivot.columns
#     ENV2_pivot         = ENV2_pivot.append(ENV2_raw[ENV2_pivot_columns].sum().rename('Grand Total')).fillna('')
    return(ENV2_pivot)


# Perform comparison of both ENV tables
def ENV_comparison(ENV1_pivot,ENV2_pivot):
    absolute_variances = ENV2_pivot.subtract(ENV1_pivot, fill_value=0)
    relative_variances = absolute_variances.div(ENV1_pivot[ENV1_pivot.columns.tolist()], fill_value=0)
    
    relative_variances = relative_variances.fillna(0)
    relative_variances = relative_variances.add_prefix('% Variance in ')
    relative_variances = relative_variances.multiply(100)
    final_variance_table = pd.concat([absolute_variances, relative_variances], axis=1)
    
    return(final_variance_table)

def grandTotalfunc(ENV1_pivot, ENV2_pivot):
    ENV1_pivot         = ENV1_pivot.append(ENV1_raw[ENV1_pivot.columns].sum().rename('Grand Total')).fillna('')
    ENV2_pivot         = ENV2_pivot.append(ENV2_raw[ENV2_pivot.columns].sum().rename('Grand Total')).fillna('')
    GT_variances = (ENV1_pivot.tail(1)-ENV2_pivot.tail(1)) / ENV1_pivot.tail(1) 
    GT_variances = GT_variances.multiply(100)
    GT_variances = GT_variances.add_prefix('% Variance in ')
    final_variance_table_TOTALS = pd.concat([(ENV1_pivot.tail(1)-ENV2_pivot.tail(1)), GT_variances], axis=1).fillna(0)
    final_variance_table_TOTALS.replace([np.inf,-np.inf], 0 , inplace=True) 
    return(final_variance_table_TOTALS)

# Determine pass/fail status for various checks
row_count_check   = []
row_count_val     = []
balance_check     = []
balance_check_val = []
def statusChecker(final_variance_table_TOTALS):
    if abs((final_variance_table_TOTALS['COUNT'].sum())) > 0:
        row_count_check.append("Fail")
    if abs((final_variance_table_TOTALS['COUNT'].sum())) == 0:
        row_count_check.append("Pass")
    
    row_count_val.append(final_variance_table_TOTALS['COUNT'].sum())

    if abs(((final_variance_table_TOTALS.drop(['COUNT'], axis=1)).values.sum())) < 5:
        balance_check.append("Pass")
    if abs(((final_variance_table_TOTALS.drop(['COUNT'], axis=1)).values.sum())) > 5:
        balance_check.append("Fail")
    
    balance_check_val.append(((final_variance_table_TOTALS.drop(['COUNT'], axis=1)).values.sum()))    

col_count_check = []
col_name_check = []
def columnChecker(ENV1_raw, ENV2_raw):   
    if len(ENV2_raw.columns) != len(ENV1_raw.columns):
        col_count_check.append("Fail")
    if len(ENV2_raw.columns) == len(ENV1_raw.columns):
        col_count_check.append("Pass")
        
    if ENV1_raw.columns.tolist() != ENV2_raw.columns.tolist():
        col_name_check.append("Fail")
    if ENV1_raw.columns.tolist() == ENV2_raw.columns.tolist():
        col_name_check.append("Pass")
        

# Saves details of all variances
details_list = []
details_extract = []
def defectDetails(final_variance_table_TOTALS, table):
    for name, values in final_variance_table_TOTALS.iteritems():
        details = ('{name}: {value}'.format(name=name, value=values[0]))
        details_extract.append(table)
        details_list.append(details)
        
        
#Loop through all tables in table_list and perform all functions
for table in range(len(table_list)): 
    ENV1_pivot = ENV1_import(sql_list[table], table_list[table], group_by_list[table])
    ENV2_pivot = ENV2_import(sql_list[table], table_list[table], group_by_list[table]) 
    

    final_variance = ENV_comparison(ENV1_pivot, ENV2_pivot)
    if table == 0: display(final_variance)
    
    final_variance_table_TOTALS = grandTotalfunc(ENV1_pivot, ENV2_pivot)
    if table == 0: display(final_variance_table_TOTALS)  
    
    statusChecker(final_variance_table_TOTALS)
    
    columnChecker(ENV1_pivot, ENV2_pivot)        
    defectDetails(final_variance_table_TOTALS, table_list[table])

#Close db connection
con.close() 


defect_description_df = pd.DataFrame(data = zip(details_extract, details_list),columns=['Table Name','Description'])      
display(defect_description_df)

results_df = pd.DataFrame(data=zip(tables_checked,row_count_check,row_count_val,balance_check, balance_check_val, col_count_check, col_name_check),
                           columns=['Table Name','Row Count Check','Row Count Variance Val','Balance Check','Balance Variance Val', 'Col Count Check', 'Col Name Check'])       
display(results_df)

FINOTC_SRC_DATA1


COUNT  balance_in_ccy1  balance_in_ccy2  \
entity_identifier group_by1                                              
231               GER            1.0           5422.0     3.400000e+00   
3600              ENG            0.0              0.0     0.000000e+00   
                  SWE            1.0      -97089201.0    -1.164153e-10   
8383              ENG            0.0          53683.0     0.000000e+00   
                  FRA            0.0        -674928.0     2.999995e-04   
93720             AUS        35255.0         432432.0     3.433243e+04   

                             % Variance in COUNT  \
entity_identifier group_by1                        
231               GER                        inf   
3600              ENG                        0.0   
                  SWE                      100.0   
8383              ENG                        0.0   
                  FRA                        0.0   
93720             AUS                        inf   

                             % Variance in balance_in_ccy1  \
entity_identifier group_by1                                  
231               GER                                  inf   
3600              ENG                             0.000000   
                  SWE                           -99.955489   
8383              ENG                          1283.668101   
                  FRA                           -99.919168   
93720             AUS                                  inf   

                             % Variance in balance_in_ccy2  
entity_identifier group_by1                                 
231               GER                                  inf  
3600              ENG                         0.000000e+00  
                  SWE                        -1.532801e-14  
8383              ENG                         0.000000e+00  
                  FRA                         8.440885e-09  
93720             AUS                                  inf

,COUNT,balance_in_ccy1,balance_in_ccy2,% Variance in COUNT,% Variance in balance_in_ccy1,% Variance in balance_in_ccy2
Grand Total,-35257.0,97272592.0,-34335.8303,-881425.0,98.98008,-0.781754


FINOTC_REP_DATA1
REPORTING_INTER_DATA


,Table Name,Description
0,FINOTC_SRC_DATA1,COUNT: -35257.0
1,FINOTC_SRC_DATA1,balance_in_ccy1: 97272592.0
2,FINOTC_SRC_DATA1,balance_in_ccy2: -34335.830299999565
3,FINOTC_SRC_DATA1,% Variance in COUNT: -881425.0
4,FINOTC_SRC_DATA1,% Variance in balance_in_ccy1: 98.98007953565528
5,FINOTC_SRC_DATA1,% Variance in balance_in_ccy2: -0.781753768984...
6,FINOTC_REP_DATA1,COUNT: 0.0
7,FINOTC_REP_DATA1,balance: 915719.0
8,FINOTC_REP_DATA1,balance2: 7837.569880000001
9,FINOTC_REP_DATA1,% Variance in COUNT: 0.0


,Table Name,Row Count Check,Row Count Variance Val,Balance Check,Balance Variance Val,Col Count Check,Col Name Check
0,FINOTC_SRC_DATA1,Fail,-35257.0,Fail,9.635693e+07,Pass,Pass
1,FINOTC_REP_DATA1,Pass,0.0,Fail,9.236754e+05,Fail,Fail
2,REPORTING_INTER_DATA,Pass,0.0,Pass,-9.209127e-13,Fail,Fail
